In [2]:
import pandas as pd
from nltk.corpus import stopwords

In [3]:
!conda install -y gdown

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::imagemagick==7.1.0_62=pl5321h0dc3a92_0
  - conda-forge/noarch::networkx==2.7=pyhd8ed1ab_0
  - conda-forge/linux-64::cartopy==0.19.0.post1=py37h0c48da3_1
  - conda-forge/linux-64::pillow==9.2.0=py37h850a105_2
  - conda-forge/linux-64::openjpeg==2.5.0=h7d73246_1
  - conda-forge/noarch::seaborn==0.12.0=hd8ed1ab_0
  - file:///tmp/conda/linux-64::dlenv-base==1.0.20220913=py37hadde398_0
  - conda-forge/noarch::visions==0.7.4=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib==3.5.3=py37h89c1867_2
  - conda-forge/linux-64::phik==0.12.2=py37h237e563_0
  - file:///tmp/conda/linux-64::dlenv-tf-2-9-gpu==1.0.20220913=py37hddb555a_0
  - conda-forge/noarch::imagehash==4.3.0=pyhd8ed1ab_0
  - conda-forge/linux-64::matplotlib-base==

In [4]:
## Laod model to kaggle
url = "https://drive.google.com/file/d/12suEakaHP4bZUQ-MfqMEqeFG1Om820pK/view?usp=sharing"
url_2 = "https://drive.google.com/file/d/1-1J2j5ezbd_TlXC3_xbc_urQjt0EBzXe/view?usp=share_link"
url_3 = "https://drive.google.com/file/d/10KtslNMXn0sHdV9eV044okAIB-fu07SK/view?usp=share_link"
!gdown --id "12suEakaHP4bZUQ-MfqMEqeFG1Om820pK"

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/opt/conda/lib/python3.7/site-packages/gdown/cli.py:125: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=12suEakaHP4bZUQ-MfqMEqeFG1Om820pK
To: /kaggle/working/model_scripted262.pt
100%|█████████████████████████████████████████| 436M/436M [00:03<00:00, 118MB/s]


In [4]:
# bert_df_1 = pd.read_csv("/kaggle/input/datasettaged20k/df_tagedforbert__5.csv")
# bert_df_2 = pd.read_csv("/kaggle/input/datasettaged20k/df_tagedforbert_5_15.csv")
# bert_df = pd.concat([bert_df_1, bert_df_2])
bert_df = pd.read_csv("/kaggle/input/20k-finalmodel/20.csv")
bert_df.head()

,Unnamed: 0,Unnamed: 0.1,sentence_id,word,word_mdf,tag
0,0,0,sentence_0,good,good,O
1,1,1,sentence_0,command,command,O
2,2,2,sentence_0,of,of,O
3,3,3,sentence_0,both,both,O
4,4,4,sentence_0,english,english,O


# some cleaning for words labels

In [5]:
## define column nedded for model
bert_df = bert_df[["sentence_id", "word", "tag"]]
bert_df.reset_index(drop=True)

,sentence_id,word,tag
0,sentence_0,good,O
1,sentence_0,command,O
2,sentence_0,of,O
3,sentence_0,both,O
4,sentence_0,english,O
...,...,...,...
551641,sentence_19999,bb,O
551642,sentence_19999,business,O
551643,sentence_19999,to,O
551644,sentence_19999,business,O


In [6]:
# untag all stopwords which taged as a skill
bert_df.reset_index(drop=True, inplace=True)
stop_words = set(stopwords.words('english'))
df_stopwords = bert_df[bert_df["word"].isin(stop_words)]
index = df_stopwords[df_stopwords["tag"] != "O"].index
bert_df.loc[index, "tag"] = "O"
#bert_df.loc[index, :]
bert_df.reset_index(drop=True)

,sentence_id,word,tag
0,sentence_0,good,O
1,sentence_0,command,O
2,sentence_0,of,O
3,sentence_0,both,O
4,sentence_0,english,O
...,...,...,...
551641,sentence_19999,bb,O
551642,sentence_19999,business,O
551643,sentence_19999,to,O
551644,sentence_19999,business,O


In [7]:
bert_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 551646 entries, 0 to 551645
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   sentence_id  551646 non-null  object
 1   word         551642 non-null  object
 2   tag          551646 non-null  object
dtypes: object(3)
memory usage: 12.6+ MB


In [8]:
bert_df[bert_df["word"].isnull()]

,sentence_id,word,tag
67468,sentence_2621,NaN,O
67469,sentence_2621,NaN,O
192879,sentence_7187,NaN,O
192880,sentence_7187,NaN,O


In [9]:
bert_df.iloc[1254:,:]

,sentence_id,word,tag
1254,sentence_62,assume,O
1255,sentence_62,every,O
1256,sentence_62,increasing,O
1257,sentence_62,responsibility,O
1258,sentence_62,in,O
...,...,...,...
551641,sentence_19999,bb,O
551642,sentence_19999,business,O
551643,sentence_19999,to,O
551644,sentence_19999,business,O


In [11]:
bert_df["tag"].value_counts()

O          482814
B-SKILL     54879
I-SKILL     12657
O-SKILL      1296
Name: tag, dtype: int64

In [12]:
!pip install transformers seqeval[gpu]

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6415184eaff61edb25e883a35c13df323ed20e8b6f80f735222d9cbad81572b4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [14]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification

In [15]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [16]:
# pandas has a very handy "forward fill" function to fill missing values based on the last upper non-nan value
bert_df = bert_df.fillna(method='ffill')
bert_df.head()

,sentence_id,word,tag
0,sentence_0,good,O
1,sentence_0,command,O
2,sentence_0,of,O
3,sentence_0,both,O
4,sentence_0,english,O


In [17]:
# let's create a new column called "sentence" which groups the words by sentence 
data = bert_df
data.rename(columns={"sentence_id": "Sentence #", "word":"Word", "tag":"Tag"}, inplace=True)
data['sentence'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Word'].transform(lambda x: ' '.join(x))
# let's also create a new column called "word_labels" which groups the tags by sentence 
data['word_labels'] = data[['Sentence #','Word','Tag']].groupby(['Sentence #'])['Tag'].transform(lambda x: ','.join(x))
data.tail()

,Sentence #,Word,Tag,sentence,word_labels
551641,sentence_19999,bb,O,complete understanding of software development...,"O,O,O,B-SKILL,I-SKILL,O,O,O,O,O,O,O,O,O,O,O,O,..."
551642,sentence_19999,business,O,complete understanding of software development...,"O,O,O,B-SKILL,I-SKILL,O,O,O,O,O,O,O,O,O,O,O,O,..."
551643,sentence_19999,to,O,complete understanding of software development...,"O,O,O,B-SKILL,I-SKILL,O,O,O,O,O,O,O,O,O,O,O,O,..."
551644,sentence_19999,business,O,complete understanding of software development...,"O,O,O,B-SKILL,I-SKILL,O,O,O,O,O,O,O,O,O,O,O,O,..."
551645,sentence_19999,applications,O,complete understanding of software development...,"O,O,O,B-SKILL,I-SKILL,O,O,O,O,O,O,O,O,O,O,O,O,..."


In [18]:
label2id = {k: v for v, k in enumerate(data.Tag.unique())}
id2label = {v: k for v, k in enumerate(data.Tag.unique())}
label2id

{'O': 0, 'B-SKILL': 1, 'I-SKILL': 2, 'O-SKILL': 3}

In [19]:
data = data[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
data.head()

,sentence,word_labels
0,good command of both english & arabic languages,"O,O,O,O,O,O,O,O"
1,must be a graduate,"O,O,O,O"
2,eriences in a development role for microsoft d...,"O,O,O,O,O,O,B-SKILL,I-SKILL"
3,solid understanding of programming languages c...,"O,O,O,O,O,O,O,O"
4,solid understating of data base concepts,"O,O,O,O,O,O"


In [20]:
data.shape

(17354, 2)

In [21]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [22]:
def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [23]:
class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        sentence = self.data.sentence[index]  
        word_labels = self.data.word_labels[index]  
        tokenized_sentence, labels = tokenize_and_preserve_labels(sentence, word_labels, self.tokenizer)
        
        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]
        
        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]
        
        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

In [24]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (17354, 2)
TRAIN Dataset: (13883, 2)
TEST Dataset: (3471, 2)


In [25]:
training_set[0]

{'ids': tensor([  101,  2478,  1996, 22889,  2050,  2565,  2000,  3113,  1996, 15117,
          1998, 12422,  1996, 10813,  8122,  1998, 10640,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [26]:
# print the first 30 tokens and corresponding labels
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["ids"][:30]), training_set[0]["targets"][:30]):
  print('{0:10}  {1}'.format(token, id2label[label.item()]))

[CLS]       O
using       O
the         O
sl          O
##a         O
program     O
to          O
meet        O
the         O
deadline    O
and         O
unsure      O
the         O
deployment  B-SKILL
efficiency  O
and         O
accuracy    O
[SEP]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O
[PAD]       O


In [27]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [28]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', 
                                                    num_labels=len(id2label),
                                                    id2label=id2label,
                                                    label2id=label2id)
#model = torch.load('/kaggle/working/model_scripted262.pt')
model.to(device)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [29]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(1.2139, device='cuda:0', grad_fn=<NllLossBackward0>)

In [30]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 4])

In [31]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [32]:
from transformers import BertForSequenceClassification, BertConfig

output_model = '/content/drive/MyDrive/Certifications/model_xlnet_mid.pth'
def save(model, optimizer):
    # save
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, output_model)

In [33]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_preds.extend(predictions)
        tr_labels.extend(targets)
        
        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    #save(model, optimizer)
    torch.save(model, '/kaggle/working//model_scripted6323.pt')

In [34]:
%%time
for epoch in range(20):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 1.2709558010101318
Training loss per 100 training steps: 0.18787004145802838
Training loss per 100 training steps: 0.1252772487069837
Training loss per 100 training steps: 0.09842278302780219
Training loss per 100 training steps: 0.08255397079470672
Training loss epoch: 0.07855395925304239
Training accuracy epoch: 0.9322280468965433
Training epoch: 2
Training loss per 100 training steps: 0.02533300593495369
Training loss per 100 training steps: 0.028124927709081975
Training loss per 100 training steps: 0.025721905780817147
Training loss per 100 training steps: 0.024103198178805584
Training loss per 100 training steps: 0.02281806006809311
Training loss epoch: 0.022610361530115048
Training accuracy epoch: 0.9745756796640919
Training epoch: 3
Training loss per 100 training steps: 0.021969251334667206
Training loss per 100 training steps: 0.015002444084493978
Training loss per 100 training steps: 0.014912371951113664
Training loss per

In [35]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)
            
            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(targets)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
    
    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [36]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.0004924497334286571
Validation loss per 100 evaluation steps: 0.007858470556345877
Validation loss per 100 evaluation steps: 0.009057898696523604
Validation Loss: 0.008946437419837007
Validation Accuracy: 0.9936946865323996


In [37]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

/opt/conda/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: O-SKILL seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


              precision    recall  f1-score   support

       SKILL       0.97      0.98      0.97     10498

   micro avg       0.97      0.98      0.97     10498
   macro avg       0.97      0.98      0.97     10498
weighted avg       0.97      0.98      0.97     10498



# Inferance

In [38]:
text = """Ahmed Ramzy Fathy Saleh                                                   Details:
Machine learning engineer on cloud                                                       phone: 01090668772
                                                                                        Date of birth: 8/9/200
                                                                                                                Military service: executed
                                                                                                   Mail:ahmed.ramzy.2000.22@gmail.com
Objectives:
To get a challenging career that can help me improve my skills & I could express myself through it, to gain more information & more skills in my professional life so I could be useful more & more for a Such a successful organization.
Employment history:
Machine learning engineer on the cloud at NTI, Nasr city   
Nov 2022-Jan 2023. 
Education:
Graduated from October 6 university faculty of computer science Department of information system (June 2022) 
• Cumulative Grade: very good 
• Graduated project: online pizza ordering system 
Training &Courses: 
• Introduction to the network (CCNA)                                                  - machine learning on cloud	
• Egypt fwd. WEB Development Challenger Track (Aug 2021)          -Natural Language Processing 
• Egypt fwd. data analytics                                                                    -Freelancing Skills (12hrs)
• Linked in Certificate of data analytics on (July 03, 2022)                  - Project Management (36hrs)
• Certification of (HCIA, 5G) ( Jan 07, 2024) 	                         	       -Soft Skills (30hrs)
• HCIA big data 
• Fundamental of data base Mahartech.gov 	        
• Oracle SQL database                                                                  
•Introduction to Data Center (36hr)                                              -	
•Programming with Python (PCAP):
•Cloud Computing Concepts
-AWS Core Services.
-AWS Security.
-AWS Architecting.
-AWS Cloud Economics.
Skills: 
• MS Office (Word, Powerpoint, excel).
•Fast learner
• Adaptability
•Ability to work on a team
•Excellent communication skills
•creative thinking
References
Upon your request either in soft or hard copy for graduation certificate, military, courses, and other certificates




"""

In [39]:
!pip install clean-text
!pip uninstall emoji -y
!pip install emoji==1.7
!pip install clean-text[gpl]

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 5.0 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=ca7228140c08d023bc6166754d8fb406133707e6fff8a6448c176248998d951a
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji
  Attempting uninstall: emoji
    Found existing installation: emoji 2.2.0
    Uninstalling emoji-2.2.0:
      Successfully uninstalled emoji-2.2.0
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Found existing installation: emoji 1.7.0
Uninstalling emoji-1.7.0:
  Successfully uninstalled emoji-1.7.0
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information ava

In [40]:
## cleaning corpus with cleantetx fun
from cleantext import clean
new_txt = clean(text, lower=True,
    fix_unicode=True,
    to_ascii=True,
    normalize_whitespace=False,
    no_line_breaks=True,
    strip_lines=True,
    keep_two_line_breaks=False,
    no_urls=True,
    no_emails=False,
    no_phone_numbers=True,
    no_numbers=False,
    no_digits=False,
    no_currency_symbols=True,
    no_punct=False,
    no_emoji=True,
    replace_with_url="<URL>",
    replace_with_email="<EMAIL>",
    replace_with_phone_number="<PHONE>",
    #replace_with_number="",
    #replace_with_digit="",
    replace_with_currency_symbol="",
    replace_with_punct=" ",
    lang="en",)
print(new_txt)

ahmed ramzy fathy saleh                                                   details:
machine learning engineer on cloud                                                       phone: <phone>
                                                                                        date of birth: 8/9/200
                                                                                                                military service: executed
                                                                                                   mail:ahmed.ramzy.2000.22@gmail.com
objectives:
to get a challenging career that can help me improve my skills & i could express myself through it, to gain more information & more skills in my professional life so i could be useful more & more for a such a successful organization.
employment history:
machine learning engineer on the cloud at nti, nasr city   
nov 2022-jan 2023. 
education:
graduated from october 6 university faculty of computer science departme

In [41]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 
def cleanhtml(new_txt):
  cleantext = re.sub(CLEANR, '', new_txt)
  return cleantext
text = cleanhtml(new_txt)

In [42]:
len(text.split())

244

In [43]:
## prediction using pytorch
from itertools import groupby

def inferance(text=text, model=model):
        MAX_LEN = 512
        id2label = {0:'O', 1:'B-SKILL', 2:'I-SKILL', 3:"O-SKILL"}
        sentence = text #"i usualy used bootstrap for web tasks and jupyter as ide for python, server sql database"

        inputs = tokenizer(sentence, padding='max_length', truncation=True, max_length=MAX_LEN, return_tensors="pt")

        # move to gpu
        model = model.to(device)
        ids = inputs["input_ids"].to(device)
        mask = inputs["attention_mask"].to(device)
        # forward pass
        outputs = model(ids, mask)
        logits = outputs[0]
        active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        score_softmax = torch.softmax(active_logits, axis=1)  ## calculate softmax for predection value
        score = torch.max(score_softmax, axis=1)              ## get max output
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level
        #score = torch.max(active_logits, axis=1)
        tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
        token_predictions = [id2label[i] for i in flattened_predictions.cpu().numpy()]
        wp_preds = list(zip(tokens, token_predictions, score[0])) # list of tuples. Each tuple = (wordpiece, prediction)
        #yhat_classes = np.where(flattened_predictions.cpu().numpy() > 0.5, 1, 0).squeeze().item()

        word_level_predictions = []
        for pair in wp_preds:
          if (pair[0].startswith("##")) or (pair[0] in ['[CLS]', '[SEP]', '[PAD]']):
            # skip prediction
            continue
          else:
            word_level_predictions.append([pair[1], pair[2].item()])

        # we join tokens, if they are not special ones
        str_rep = " ".join([t[0] for t in wp_preds if t[0] not in ['[CLS]', '[SEP]', '[PAD]']]).replace(" ##", "")
        #print(f"sentnce length:- {len(str_rep.split())}", f"[{str_rep}]", sep="\n")
        #print(f"predicted_label len:- {len(word_level_predictions)}", word_level_predictions, sep="\n")
        f_ls = list(zip(str_rep.split(), word_level_predictions))
        skills = [( x[0], x[1][0], x[1][1]) for x in f_ls if x[1][0] == "B-SKILL" or x[1][0] == "I-SKILL" or x[1][0] == "O-SKILL"]
        #print(yhat_classes)
        skills = [next(g) for _, g in groupby(skills, key=lambda x:x[0])]

        #print(skills, end="\n")
        return skills, tokens

In [44]:
# split resume in sentences
skills = []
tokens = []
for sub_tex in text.split("."):
    skill_line, token_line = inferance(text=sub_tex)
    skills.extend(skill_line)
    tokens.extend(token_line)
print(skills)

[('machine', 'B-SKILL', 0.9999083280563354), ('learning', 'I-SKILL', 0.9998332262039185), ('machine', 'B-SKILL', 0.9999176263809204), ('learning', 'I-SKILL', 0.9998291730880737), ('computer', 'B-SKILL', 0.999924898147583), ('science', 'I-SKILL', 0.9995020627975464), ('machine', 'B-SKILL', 0.9999222755432129), ('learning', 'I-SKILL', 0.9998760223388672), ('natural', 'B-SKILL', 0.9993398785591125), ('language', 'I-SKILL', 0.996733546257019), ('processing', 'O-SKILL', 0.9983715415000916), ('analytics', 'B-SKILL', 0.999797523021698), ('certificate', 'B-SKILL', 0.9999103546142578), ('analytics', 'B-SKILL', 0.9998558759689331), ('project', 'B-SKILL', 0.9999485015869141), ('management', 'I-SKILL', 0.9999059438705444), ('big', 'B-SKILL', 0.9999476671218872), ('data', 'I-SKILL', 0.9998098015785217), ('oracle', 'B-SKILL', 0.9999568462371826), ('sql', 'I-SKILL', 0.9997723698616028), ('database', 'B-SKILL', 0.9998849630355835), ('data', 'B-SKILL', 0.9997982382774353), ('center', 'I-SKILL', 0.99980

In [45]:
import itertools
def join_tokens(tokens=tokens):
    res = ''
    if tokens:
        res = tokens[0]
        for token in tokens[1:]:
            if not (token.isalpha() and res[-1].isalpha()):
                res += token  # punctuation
            else:
                res += ' ' + token  # regular word
    return res
def collapse(skills):
    # List with the result
    ner_result = skills
    collapsed_result = []


    current_entity_tokens = []
    current_entity = None

    # Iterate over the tagged tokens
    for token, tag, score in ner_result:

        if tag.startswith("B-"):
            # ... if we have a previous entity in the buffer, store it in the result list
            if current_entity is not None:
                collapsed_result.append([join_tokens(current_entity_tokens), current_entity])

            current_entity = tag[2:]
            # The new entity has so far only one token
            current_entity_tokens = [token]

        # If the entity continues ...
        elif current_entity_tokens!= None and tag == "I-" + str(current_entity):
            # Just add the token buffer
            current_entity_tokens.append(token)
        elif current_entity_tokens!= None and tag == "O-" + str(current_entity):
            # Just add the token buffer
            current_entity_tokens.append(token)
        else:
            collapsed_result.append([join_tokens(current_entity_tokens), current_entity])
            collapsed_result.append([token,tag[2:]])

            current_entity_tokens = []
            current_entity = None

            pass

    # The last entity is still in the buffer, so add it to the result
    # ... but only if there were some entity at all
    if current_entity is not None:
        collapsed_result.append([join_tokens(current_entity_tokens), current_entity])
        collapsed_result = sorted(collapsed_result)
        collapsed_result = list(k for k, _ in groupby(collapsed_result))

    return collapsed_result
collapsed_result = collapse(skills)
collapsed_result =  dict(collapsed_result)
for entity in collapsed_result.keys():
    for skill in skills:
        if skill[0] in entity.split()[-1]:
            collapsed_result[entity] = "SKILL" +", score: "+ "{:.2f}".format(skill[2])
print(collapsed_result)
            # collapsed_result

{'analytics': 'SKILL, score: 1.00', 'aws': 'SKILL, score: 1.00', 'big data': 'SKILL, score: 1.00', 'certificate': 'SKILL, score: 1.00', 'computer science': 'SKILL, score: 1.00', 'data center': 'SKILL, score: 1.00', 'database': 'SKILL, score: 1.00', 'machine learning': 'SKILL, score: 1.00', 'natural language processing': 'SKILL, score: 1.00', 'oracle sql': 'SKILL, score: 1.00', 'project management': 'SKILL, score: 1.00', 'python': 'SKILL, score: 1.00'}


In [46]:
%%time
from transformers import pipeline
token = []
for sub_tex in text.split("."):
    pipe = pipeline(task="ner",model=model.to('cpu'),tokenizer=tokenizer, aggregation_strategy="simple")
    #r"I usualy used bootstrap for web tasks and jupyter as ide for python, server sql"
    token.extend(pipe(sub_tex))
#print(token)
token_md = token
entities = []
for row, entity in enumerate(token_md):
    if re.search("#", entity["word"]) != None:
        word = re.sub("#", "", entity["word"])
        #word = entity["word"]
        entities[-1]["word"] = entities[-1]["word"] + word
        #token_md.remove(entity)
    else:
        entities.append(entity)
print(entities)

[{'entity_group': 'SKILL', 'score': 0.9998708, 'word': 'machine learning', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.9998734, 'word': 'machine learning', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.99971354, 'word': 'computer science', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.99989915, 'word': 'machine learning', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.99803674, 'word': 'natural language', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.99837154, 'word': 'processing', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.9997975, 'word': 'analytics', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.9999105, 'word': 'certificate', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.9998559, 'word': 'analytics', 'start': None, 'end': None}, {'entity_group': 'SKILL', 'score': 0.9999273, 'word': 'project management', 'start': None, 'e

In [47]:
## saving bert model to run on local machine
import os

directory = "/kaggle/working//model"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed
